In [15]:
from lxml.html import parse
from urllib.request import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options?ltr=1'))

doc = parsed.getroot()

In [64]:
# table
tables = doc.findall('.//table')

In [76]:
tables

[<Element table at 0x234b3aae0e8>, <Element table at 0x234b3aae278>]

In [66]:
# tables의 tag, attrib 반환
for elem in tables:
    print(elem.tag)
    print(elem.attrib)

table
{'class': 'calls table-bordered W(100%) Pos(r) Bd(0) Pt(0) list-options', 'data-reactid': '23'}
table
{'class': 'puts table-bordered W(100%) Pos(r) list-options', 'data-reactid': '697'}


In [18]:
calls = tables[0]
calls

<Element table at 0x234b3aae0e8>

In [40]:
puts = tables[1]
puts

<Element table at 0x234b3aae278>

In [21]:
calls.text_content()

'Contract NameLast Trade DateStrikeLast PriceBidAskChange% ChangeVolumeOpen InterestImplied VolatilityAAPL190322C001200002019-03-19 3:31PM EDT120.0066.400.000.000.00-200.00%AAPL190322C001250002019-03-19 3:31PM EDT125.0061.420.000.000.00-200.00%AAPL190322C001300002019-03-19 3:31PM EDT130.0056.750.000.000.00-200.00%AAPL190322C001350002019-03-19 3:31PM EDT135.0051.490.000.000.00-100.00%AAPL190322C001400002019-03-08 1:10PM EDT140.0031.800.000.000.00-600.00%AAPL190322C001450002019-03-05 6:53PM EDT145.0030.9540.8041.700.00-10100.00%AAPL190322C001460002019-03-07 11:11AM EDT146.0027.200.000.000.00-1000.00%AAPL190322C001470002019-03-11 10:50AM EDT147.0031.120.000.000.00-100.00%AAPL190322C001480002019-03-20 10:55AM EDT148.0039.200.000.000.00-100.00%AAPL190322C001490002019-02-07 1:44PM EDT149.0022.3336.6537.700.00-0410.00%AAPL190322C001500002019-03-20 11:54AM EDT150.0036.590.000.000.00-600.00%AAPL190322C001525002019-03-20 1:59PM EDT152.5034.600.000.000.00-400.00%AAPL190322C001550002019-03-15 3:58

- row[0]는 './/th' 로 구분되고
- row[1:]는 전부 './/td'로 구분되어 있다.

In [22]:
def _unpack(row, tag='td'):
    tds = row.findall('.//%s' % tag)
    return [td.text_content() for td in tds] # []로 마지막에 감싸줌.

In [85]:
_unpack(calls[0], tag = 'th')

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [87]:
_unpack(calls, tag = 'td')

['AAPL190322C00120000',
 '2019-03-19 3:31PM EDT',
 '120.00',
 '66.40',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '2',
 '0',
 '0.00%',
 'AAPL190322C00125000',
 '2019-03-19 3:31PM EDT',
 '125.00',
 '61.42',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '2',
 '0',
 '0.00%',
 'AAPL190322C00130000',
 '2019-03-19 3:31PM EDT',
 '130.00',
 '56.75',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '2',
 '0',
 '0.00%',
 'AAPL190322C00135000',
 '2019-03-19 3:31PM EDT',
 '135.00',
 '51.49',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '1',
 '0',
 '0.00%',
 'AAPL190322C00140000',
 '2019-03-08 1:10PM EDT',
 '140.00',
 '31.80',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '6',
 '0',
 '0.00%',
 'AAPL190322C00145000',
 '2019-03-05 6:53PM EDT',
 '145.00',
 '30.95',
 '40.80',
 '41.70',
 '0.00',
 '-',
 '10',
 '10',
 '0.00%',
 'AAPL190322C00146000',
 '2019-03-07 11:11AM EDT',
 '146.00',
 '27.20',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '10',
 '0',
 '0.00%',
 'AAPL190322C00147000',
 '2019-03-11 10:50AM EDT',
 '147.00',
 '31.12',
 '0.00',
 '0.00',
 '0.00',
 '-',
 '1',
 

In [35]:
import pandas as pd

In [92]:
# TextParser.get_chunk() : 들어오는 데이터들을 type에 따라 개별적으로 분해해줌.
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr') 
    header = _unpack(rows[0], tag='th') # row[0] : col_name들을 모아놓은 행
    data = [_unpack(row) for row in rows[1:]] # row[1:] : 모든 record, 그리고 data에는 [] 한 번 더 감싸준다.
    return TextParser(data, names=header).get_chunk() # get_chunk() : get_DataFrame이라고 생각하면 됨.

In [93]:
data = [_unpack(call) for call in calls[1:]] # ?_?
data[:20]

[['AAPL190322C00120000',
  '2019-03-19 3:31PM EDT',
  '120.00',
  '66.40',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '2',
  '0',
  '0.00%',
  'AAPL190322C00125000',
  '2019-03-19 3:31PM EDT',
  '125.00',
  '61.42',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '2',
  '0',
  '0.00%',
  'AAPL190322C00130000',
  '2019-03-19 3:31PM EDT',
  '130.00',
  '56.75',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '2',
  '0',
  '0.00%',
  'AAPL190322C00135000',
  '2019-03-19 3:31PM EDT',
  '135.00',
  '51.49',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '1',
  '0',
  '0.00%',
  'AAPL190322C00140000',
  '2019-03-08 1:10PM EDT',
  '140.00',
  '31.80',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '6',
  '0',
  '0.00%',
  'AAPL190322C00145000',
  '2019-03-05 6:53PM EDT',
  '145.00',
  '30.95',
  '40.80',
  '41.70',
  '0.00',
  '-',
  '10',
  '10',
  '0.00%',
  'AAPL190322C00146000',
  '2019-03-07 11:11AM EDT',
  '146.00',
  '27.20',
  '0.00',
  '0.00',
  '0.00',
  '-',
  '10',
  '0',
  '0.00%',
  'AAPL190322C00147000',
  '2019-03

In [38]:
dfc = parse_options_data(calls) # calls = table[0]

In [39]:
dfp = parse_options_data(puts) # puts = table[1]

In [12]:
# 1. 수직 결합으로 multi index로 만들자. call, put

In [47]:
result1 = pd.concat([dfc, dfp], keys=['calls', 'puts'])
result1[:5]

Contract Name        Last Trade Date  Strike  Last Price  Bid  \
calls 0  AAPL190322C00120000  2019-03-19 3:31PM EDT   120.0       66.40  0.0   
      1  AAPL190322C00125000  2019-03-19 3:31PM EDT   125.0       61.42  0.0   
      2  AAPL190322C00130000  2019-03-19 3:31PM EDT   130.0       56.75  0.0   
      3  AAPL190322C00135000  2019-03-19 3:31PM EDT   135.0       51.49  0.0   
      4  AAPL190322C00140000  2019-03-08 1:10PM EDT   140.0       31.80  0.0   

         Ask  Change % Change Volume  Open Interest Implied Volatility  
calls 0  0.0     0.0        -      2              0              0.00%  
      1  0.0     0.0        -      2              0              0.00%  
      2  0.0     0.0        -      2              0              0.00%  
      3  0.0     0.0        -      1              0              0.00%  
      4  0.0     0.0        -      6              0              0.00%

In [ ]:
# 2. 수평 결합으로 call, put 만들자 (이때, column에 Multi index를 적용해서 level 0에 calls, puts 넣어보자.)
# join, merge는 합칠 데이터가 동일해지면 열의 이름이 동일하기 때문에 적절하지 않다.
# 같은 객체에 대해 Multi indexing을 하려면 수평 결합도 그냥 pd.concat()으로 붙이자.

In [86]:
result2 = pd.concat([dfc, dfp], axis = 1, keys=['calls', 'puts'])
result2[:5]

calls                                                     \
         Contract Name        Last Trade Date Strike Last Price  Bid  Ask   
0  AAPL190322C00120000  2019-03-19 3:31PM EDT  120.0      66.40  0.0  0.0   
1  AAPL190322C00125000  2019-03-19 3:31PM EDT  125.0      61.42  0.0  0.0   
2  AAPL190322C00130000  2019-03-19 3:31PM EDT  130.0      56.75  0.0  0.0   
3  AAPL190322C00135000  2019-03-19 3:31PM EDT  135.0      51.49  0.0  0.0   
4  AAPL190322C00140000  2019-03-08 1:10PM EDT  140.0      31.80  0.0  0.0   

                                        ...                    puts         \
  Change % Change Volume Open Interest  ...         Last Trade Date Strike   
0    0.0        -      2             0  ...   2019-03-05 2:25PM EDT  120.0   
1    0.0        -      2             0  ...  2019-03-04 11:54AM EDT  125.0   
2    0.0        -      2             0  ...  2019-03-11 11:41AM EDT  130.0   
3    0.0        -      1             0  ...   2019-03-12 3:13PM EDT  135.0   
4    0.0        -      6             0  ...  2019-03-13 10:49AM EDT  140.0   

                                                              \
  Last Price  Bid   Ask Change % Change Volume Open Interest   
0       0.01  0.0  0.01    0.0        -      1           7.0   
1       0.02  0.0  0.01    0.0        -      5           5.0   
2       0.01  0.0  0.00    0.0        -      1           0.0   
3       0.01  0.0  0.00    0.0        -      5           0.0   
4       0.01  0.0  0.00    0.0        -    243           0.0   

                      
  Implied Volatility  
0            193.75%  
1            175.00%  
2             50.00%  
3             50.00%  
4             50.00%  

[5 rows x 22 columns]